<a href="https://colab.research.google.com/github/SonnyDev/drepa_ai/blob/main/Mistral_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

'/root/mistral_models/7B-Instruct-v0.3'

In [ ]:
!pip install mistral_inference mistral_common

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.9/703.9 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 82.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117029 sha256=c98bd12dd90c59208afd184c0f68be9d58b62127e9130a899f89ccdde60026ee
  Stored in directory: /root/.cache/pip/wheels/d6/6d/5d/5b73fa0f46d01a793713f8859201361e9e581ced8c75e5c6a3
Successfully built fire
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.2
    Uninstall

In [ ]:
from mistral_inference.model import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest


tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
model = Transformer.from_folder(mistral_models_path)

completion_request = ChatCompletionRequest(messages=[UserMessage(content="What is the consequence of replacing an adenine with a thymine in the DNA strand corresponding to the glutamate codon")])

tokens = tokenizer.encode_chat_completion(completion_request).tokens

out_tokens, _ = generate([tokens], model, max_tokens=64, temperature=0.0, eos_id=tokenizer.instruct_tokenizer.tokenizer.eos_id)
result = tokenizer.instruct_tokenizer.tokenizer.decode(out_tokens[0])

print(result)


The DNA codon for the amino acid glutamate is GAA or GAG. Replacing an adenine (A) with a thymine (T) would not occur in DNA, as DNA strands have adenine paired with thymine (A-T), and


In [ ]:
### Ablation study

# Question 1. What are some of the challenges faced by sickle cells (drépanocytes) in the bloodstream
# Question 2. What is the consequence of replacing an adenine with a thymine in the DNA strand corresponding to the glutamate codon


In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
dataset_name = "sonny-dev/drepa_instruct_dataset"
base_model = "mistralai/Mistral-7B-Instruct-v0.3"
new_model = "mistral_7b_drepa"

In [ ]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train")

In [ ]:
dataset["messages"][0]

[{'content': "What is sickle cell disease and how does it affect the delivery of oxygen to the body's tissues?",
  'role': 'user'},
 {'content': 'Sickle cell disease is an inherited blood disorder marked by flawed hemoglobin. It interferes with the delivery of oxygen to the tissues because red blood cells with sickle cell hemoglobin are stiff, sticky, and form into a sickle or crescent shape when they lose their oxygen. These cells stick together, block small blood vessels, and impede the movement of healthy, normal oxygen-carrying blood, causing pain.',
  'role': 'assistant'}]

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)
model.config.use_cache = False # silence the warnings
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

(True, True)

In [ ]:
# Prepare the model for k-bit training
model = prepare_model_for_kbit_training(model)

# Create a PEFT configuration with Lora
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",  # Use the appropriate task type
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"]
)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 85065728 || all params: 3843428352 || trainable%: 2.213277319342624


In [ ]:
import transformers
from datetime import datetime

project = "chat-doctor-finetune"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=500,
        learning_rate=2.5e-4, # Want a small lr for finetuning
        #bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=25,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=25,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=25,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)


In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train(resume_from_checkpoint=True)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
)

In [ ]:
import pandas as pd

In [ ]:
# Create a DataFrame
df = pd.DataFrame(dataset)

In [ ]:
def concatenate_messages(messages):
    conversation = ""
    for msg in messages:
        conversation += f"{msg['role']}: {msg['content']} "
    return conversation.strip()

# Apply the function to the messages column
df['text'] = df['messages'].apply(concatenate_messages)

# Check the processed DataFrame
print(df['text'].head())

0    user: What is sickle cell disease and how does...
1    user: How is sickle cell disease inherited? as...
2    user: How is sickle cell disease inherited? as...
3    user: What is the chance of parents who are ca...
4    user: What are some symptoms and complications...
Name: text, dtype: object


In [ ]:

# Function to extract user inputs and assistant responses
def extract_conversations(messages):
    user_inputs = []
    assistant_responses = []
    for conversation in messages:
        if isinstance(conversation, list):
            print("Processing conversation:", conversation)
            for i in range(len(conversation) - 1):
                try:
                    if conversation[i]['role'] == 'user' and conversation[i + 1]['role'] == 'assistant':
                        user_inputs.append(conversation[i]['content'])
                        assistant_responses.append(conversation[i + 1]['content'])
                except KeyError as e:
                    print(f"KeyError encountered: {e} in message: {conversation[i]}")
        else:
            print(f"Unexpected data format: {conversation}")
    return user_inputs, assistant_responses

# Apply the function to the messages column
user_inputs, assistant_responses = extract_conversations(df['messages'])

Processing conversation: [{'content': "What is sickle cell disease and how does it affect the delivery of oxygen to the body's tissues?", 'role': 'user'}, {'content': 'Sickle cell disease is an inherited blood disorder marked by flawed hemoglobin. It interferes with the delivery of oxygen to the tissues because red blood cells with sickle cell hemoglobin are stiff, sticky, and form into a sickle or crescent shape when they lose their oxygen. These cells stick together, block small blood vessels, and impede the movement of healthy, normal oxygen-carrying blood, causing pain.', 'role': 'assistant'}]
Processing conversation: [{'content': 'How is sickle cell disease inherited?', 'role': 'user'}, {'content': 'Sickle cell disease is inherited when a person receives a defective gene from both their mother and father.', 'role': 'assistant'}]
Processing conversation: [{'content': 'How is sickle cell disease inherited?', 'role': 'user'}, {'content': 'Sickle cell disease is inherited when a perso

In [ ]:
# Tokenize the inputs and responses
inputs = tokenizer(user_inputs, return_tensors="pt", padding=True, truncation=True)
labels = tokenizer(assistant_responses, return_tensors="pt", padding=True, truncation=True)

# Create a custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]
        return item

    def __len__(self):
        return len(self.labels['input_ids'])

# Convert the tokenized data into a custom dataset
train_dataset = CustomDataset(inputs, labels)


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
train_dataset[0]

{'input_ids': tensor([    1,  2592,  1117,  8714,  1059,  4369,  8798,  1072,  1678,  2003,
          1146,  5965,  1040, 10330,  1070, 21826,  1066,  1040,  2955, 29510,
         29481,  1029, 20242, 29572,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]),
 'labels': tensor([    1,  1086,  1449,  1059,  4369,  8798,  1117,  1164, 23761,  5010,
        